In [ ]:
def generate_ckd_table(site):
#site='MCRI'
#if True:

    print('Running ckd_generate on site '+site, flush = True)            

    ext = {'MCW':'dsv', 'UofU':'txt', 'UTSW':'dsv'}
    sep = {'MCW':'|',   'UofU':'|',   'UTSW':'|'}    

    print('Running staging on site '+site, flush = True)            

    if site in ext.keys():       
        labcsv=pd.read_csv(home_dir+site+'/raw/AKI_LAB.'+ext[site],sep=sep[site])
        labcsv.columns = [y.upper() for y in labcsv.columns]        
        onsetcsv=pd.read_csv(home_dir+site+'/raw/AKI_ONSETS.'+ext[site],sep=sep[site])        
        onsetcsv.columns = [y.upper() for y in onsetcsv.columns]                
    elif site =='UPITT':
        labcsv=pd.read_csv(home_dir+site+'/raw/AKI_LAB.csv', encoding='windows-1252')    
        labcsv.columns = [y.upper() for y in labcsv.columns]            
        onsetcsv=pd.read_csv(home_dir+site+'/raw/AKI_ONSETS.csv')    
        onsetcsv.columns = [y.upper() for y in onsetcsv.columns]        
    else:
        labcsv=pd.read_csv(home_dir+site+'/raw/AKI_LAB.csv')    
        labcsv.columns = [y.upper() for y in labcsv.columns]        
        onsetcsv=pd.read_csv(home_dir+site+'/raw/AKI_ONSETS.csv')    
        onsetcsv.columns = [y.upper() for y in onsetcsv.columns]        

    if site == 'UPITT':        
        labcsv['SPECIMEN_DATE_TIME'] = pd.to_datetime(labcsv['SPECIMEN_DATE_TIME'], format='%Y:%m:%d %H:%M')
    else:
        labcsv['SPECIMEN_DATE_TIME'] = pd.to_datetime(labcsv['SPECIMEN_DATE_TIME'])            

    if site == 'UPITT':        
        labcsv['LAB_ORDER_DATE'] = pd.to_datetime(labcsv['LAB_ORDER_DATE'], format='%m/%d/%Y')
    else:
        labcsv['LAB_ORDER_DATE'] = pd.to_datetime(labcsv['LAB_ORDER_DATE'])                

    onset = pd.read_pickle('data/'+site+'/p0_onset_'+site+'.pkl')        
    lab = pd.read_pickle('data/'+site+'/p0_lab_'+site+'.pkl')
    px = pd.read_pickle('data/'+site+'/p0_px_'+site+'.pkl')


    scr = labcsv[labcsv['LAB_LOINC'].isin(['2160-0','38483-4','14682-9','21232-4','35203-9','44784-7','59826-8'])]
    #    scr = lab[lab['LAB_LOINC'].isin(['2160-0','38483-4','14682-9','21232-4','35203-9','44784-7','59826-8'])]
    scr = scr[scr['RESULT_UNIT'].str.upper().isin(['MG/DL','MG'])]
    scr = scr[scr['RESULT_NUM']>0]
    scr = scr[scr['SPECIMEN_SOURCE']!='URINE']


    demo = pd.read_pickle('data/'+site+'/p0_demo_'+site+'.pkl')

    efgr = scr.copy()
    efgr = efgr[['PATID','ENCOUNTERID','LAB_ORDER_DATE','SPECIMEN_DATE_TIME','RESULT_DATE_TIME','RESULT_NUM']].groupby(['PATID', 'ENCOUNTERID','LAB_ORDER_DATE','SPECIMEN_DATE_TIME','RESULT_DATE_TIME']).mean().reset_index()
    efgr = efgr.loc[efgr.groupby(['PATID','ENCOUNTERID'])['LAB_ORDER_DATE'].idxmin()]
    efgr = efgr.astype({'PATID':'string', 'ENCOUNTERID':'string'})
    efgr = efgr.astype({'PATID':'object', 'ENCOUNTERID':'object'})

    efgr2 = efgr.merge(demo, left_on=['PATID', 'ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='left')
    efgr2['SEX_CODE'] = np.where(efgr2['SEX']=='F', 1, 0)
    efgr2['AFRICAN_CODE'] = np.where(efgr2['RACE']=='03', 1, 0)
    efgr2['EGFR'] = 175*np.power(efgr2['RESULT_NUM'],-1.154)*np.power(efgr2['AGE'],-0.203)*(0.742*efgr2['SEX_CODE']+(1-efgr2['SEX_CODE']))*(1.212*efgr2['AFRICAN_CODE']+(1-efgr2['AFRICAN_CODE']))
    efgr2['CKD_group'] = np.where(efgr2['EGFR']>90, 1,
                         np.where(efgr2['EGFR']>60, 2,
                         np.where(efgr2['EGFR']>30, 3, 4)))
    efgr2 = efgr2[['PATID', 'ENCOUNTERID', 'EGFR', 'CKD_group']]

    efgr2.to_pickle('data/'+site+'/p0_'+'ckdgroup'+'_'+site+'.pkl')

In [ ]:
site = 'UTHSCSA'
onset = pd.read_pickle('data/'+site+'/p0_onset_'+site+'.pkl')        
onsetcsv = pd.read_csv(home_dir+site+'/raw/AKI_ONSETS.csv')

In [ ]:
onset0 = onsetcsv[onsetcsv['NONAKI_ANCHOR'].notnull()]
onset1 = onsetcsv[np.logical_and(onsetcsv['AKI1_ONSET'].notnull(), np.logical_and(onsetcsv['AKI2_ONSET'].isnull(), onsetcsv['AKI3_ONSET'].isnull()))]
onset2 = onsetcsv[np.logical_and(onsetcsv['AKI2_ONSET'].notnull(), onsetcsv['AKI3_ONSET'].isnull())]
onset3 = onsetcsv[onsetcsv['AKI3_ONSET'].notnull()]

In [ ]:
onsetcsv.dtypes

In [ ]:
onsetX = onset1[['SERUM_CREAT_BASE', 'AKI1_SCR', 'AKI1_INC']].copy()

In [ ]:
onsetX['SCR_DIFF'] = (onsetX['AKI1_SCR']-onsetX['SERUM_CREAT_BASE']).copy()
onsetX['SCR_R'] = (onsetX['AKI1_SCR']/onsetX['SERUM_CREAT_BASE']).copy()

In [ ]:
onsetX